In [1]:
import numpy as np
import pandas as pd
import joblib

In [2]:
# Load the saved splits
X_test = joblib.load('X_test.pkl')
y_test = joblib.load('y_test.pkl')

In [3]:
X_test.head

<bound method NDFrame.head of        HomeWinStreak  AwayWinStreak  EFG_AWAY  EFG_HOME  FG_PCT_HOME  \
15073              3              0  0.390110  0.558442        0.506   
4338               3              0  0.459302  0.575000        0.525   
1981               0              1  0.458333  0.563830        0.521   
9119               0              2  0.608434  0.673077        0.603   
5562               0              0  0.614943  0.613333        0.547   
...              ...            ...       ...       ...          ...   
10057              0              0  0.494565  0.726744        0.651   
1740               1              1  0.511494  0.506098        0.439   
1957               1              0  0.434783  0.483516        0.462   
16635              1              0  0.500000  0.421053        0.368   
11505              1              0  0.415730  0.488636        0.466   

       FG_PCT_AWAY  HomeSpread_AtClose  AwayML_Prob  HomeML_Prob  \
15073        0.363               -11.

In [4]:
y_test.head

<bound method NDFrame.head of 15073    1
4338     1
1981     1
9119     1
5562     0
        ..
10057    1
1740     0
1957     1
16635    0
11505    1
Name: HomeWin, Length: 3423, dtype: int64>

In [5]:
# Load the saved logistic regression model
log_model = joblib.load('logistic_regression_model.pkl')

In [6]:
# Generate predictions using the model on the test data
predicted_probabilities = log_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class

In [7]:
# Combine predictions and actual outcomes into a DataFrame
df = pd.DataFrame({
    'predicted_prob': predicted_probabilities,  # Model's predicted probabilities
    'actual_result': y_test  # Actual outcomes from y_test
})

In [29]:
# Function to simulate betting based on model predictions
def evaluate_bets(predictions, actual_outcomes, risk_type='low', cutoff=0.7):
    outcomes = []
    
    if risk_type == 'low':
        # For low-risk bets, focus on high-confidence predictions
        selected_bets = predictions > cutoff
    else:
        # For high-risk bets, focus on low-confidence predictions
        selected_bets = predictions < (1 - cutoff)
    
    for bet, outcome in zip(selected_bets, actual_outcomes):
        if bet:
            # Win if the prediction is correct
            if outcome == 1:
                outcomes.append(1)  # Bet won
            else:
                outcomes.append(-1)  # Bet lost
        else:
            outcomes.append(0)  # No bet placed
    
    return np.array(outcomes)

# Evaluate low-risk betting scenario
low_risk_outcomes = evaluate_bets(df['predicted_prob'], df['actual_result'], risk_type='low', cutoff=0.7)
# Evaluate high-risk betting scenario
high_risk_outcomes = evaluate_bets(df['predicted_prob'], df['actual_result'], risk_type='high', cutoff=0.7)

# Count the number of bets for low-risk and high-risk scenarios
low_risk_bets = np.count_nonzero(low_risk_outcomes != 0)
high_risk_bets = np.count_nonzero(high_risk_outcomes != 0)

# Count the number of wins for low-risk and high-risk scenarios
low_risk_wins = np.count_nonzero(low_risk_outcomes == 1)
high_risk_wins = np.count_nonzero(high_risk_outcomes == 1)

# Calculate success rates as Wins / Number of Bets
low_risk_success = low_risk_wins / low_risk_bets if low_risk_bets > 0 else 0
high_risk_success = high_risk_wins / high_risk_bets if high_risk_bets > 0 else 0

# Calculate profitability
low_risk_profit = np.sum(low_risk_outcomes)
high_risk_profit = np.sum(high_risk_outcomes)

print(f"Low-Risk Success Rate: {low_risk_success * 100:.2f}%")
print(f"Low-Risk Profitability: {low_risk_profit}")

print(f"High-Risk Success Rate: {high_risk_success * 100:.2f}%")
print(f"High-Risk Profitability: {high_risk_profit}")

print(f"Number of Low-Risk Bets: {low_risk_bets}")
print(f"Number of High-Risk Bets: {high_risk_bets}")

print(f"Number of Low-Risk Wins: {low_risk_wins}")
print(f"Number of High-Risk Wins: {high_risk_wins}")

Low-Risk Success Rate: 95.41%
Low-Risk Profitability: 1623
High-Risk Success Rate: 5.14%
High-Risk Profitability: -1013
Number of Low-Risk Bets: 1787
Number of High-Risk Bets: 1129
Number of Low-Risk Wins: 1705
Number of High-Risk Wins: 58
